In [2]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from os import listdir
from os.path import isfile
import math
from prepare_data import *

from datetime import datetime
%load_ext tensorboard

In [3]:
dataset = Dataset.ninety
train_gen, val_gen, test_gen = prep_data(dataset, True, 4)

Dataset.ninety


In [15]:
# Model 1
base_model1 = InceptionV3(weights="imagenet")
for layer in base_model1.layers:
    layer._name += "_1"
    layer.trainable = True

# Model 2
base_model2 = InceptionV3(weights="imagenet")
for layer in base_model2.layers:
    layer._name += "_2"
    layer.trainable = True

# Model 3
base_model3 = InceptionV3(weights="imagenet")
for layer in base_model3.layers:
    layer._name += "_3"
    layer.trainable = True

In [16]:
out_layer = keras.layers.concatenate([base_model1.layers[-2].output, base_model2.layers[-2].output, base_model3.layers[-2].output])
predictions = keras.layers.Dense(291, activation='softmax')(out_layer)

model = keras.Model(inputs=[base_model1.input, base_model2.input, base_model3.input], outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
logdir = "logs/3model_ensemb_shared_concat" + "_" + dataset + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [18]:
model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback], epochs=5)

Epoch 1/5
318/318 [==============================] - 141s 370ms/step - loss: 5.0010 - accuracy: 0.0095 - val_loss: 126678.9922 - val_accuracy: 0.0077
Epoch 2/5
318/318 [==============================] - 76s 239ms/step - loss: 4.7004 - accuracy: 0.0110 - val_loss: 41.6525 - val_accuracy: 0.0077
Epoch 3/5
318/318 [==============================] - 76s 240ms/step - loss: 4.7136 - accuracy: 0.0126 - val_loss: 251.9150 - val_accuracy: 0.0115
Epoch 4/5
318/318 [==============================] - 76s 238ms/step - loss: 4.6387 - accuracy: 0.0118 - val_loss: 90.4941 - val_accuracy: 0.0038
Epoch 5/5
318/318 [==============================] - 76s 239ms/step - loss: 4.6047 - accuracy: 0.0118 - val_loss: 7.7261 - val_accuracy: 0.0153


In [19]:
model.evaluate(test_gen)

68/68 [==============================] - 10s 143ms/step - loss: 7.5129 - accuracy: 0.0222


[7.512945175170898, 0.02222222276031971]